# Run: Diamond Vert + Viro + Mamm run

```
Lead     : ababaian
Issue    : 
Version  : v0.3.4 dev-diamond branch
start    : 2020 07 11
complete : 2020 07 12
files    : ~/serratus/notebook/200711_ab/
s3_files : s3://serratus-public/notebook/200711_ab/
output   : s3://serratus-public/out/200711_pvert3/
```

### Intro/Objectives

- Pilot diamond run was successful, expand to all vertebrates, mammals and virome samples
- Same as v1 for set-up except protref updated to version 2 via RCE


### Update Protein Reference

In [ ]:
# Add deltaviruses to protein search
# Created protref1B locally
# ON EC2

git clone -b dev-diamond https://github.com/ababaian/serratus.git; cd serratus/containers
./build_containers.sh

sudo docker run --rm --entrypoint /bin/bash -it serratus-align:latest

# Download previous protref
# removed duplicate entries
aws s3 cp aws s3 cp s3://serratus-public/rce/protrefm_v3/protrefm_v3.fa ./protref3.fa

diamond makedb --in protref3.fa -d protref3

samtools faidx protref3.fa 
mv protref3.fa.fai protref3.sumzer.tsv

md5sum * > protref3.md5

# Upload to S3
aws s3 sync ./ s3://serratus-public/seq/protref3/

```
87623dfa5b462d3deefe845c3eeec5e4  protref3.dmnd
c4a7027dcc852d35387d32e1e22f89a0  protref3.fa
be4d0cb57b6a1e3fe4e411aaaaeacc86  protref3.sumzer.tsv
```

### Initialize local workspace

In [1]:
# Serratus commit version
SERRATUS="/home/artem/serratus"
cd $SERRATUS

# Create local run directory
WORK="$SERRATUS/notebook/200706B_ab"
mkdir -p $WORK; cd $WORK

# S3 notebook path
S3_WORK='s3://serratus-public/notebook/200706B_ab/'

# date and version
date
git rev-parse HEAD # commit version

Sat Jul 11 01:24:26 PDT 2020
672db1bdf61632554e05f13b8632dec265f91861


### SRA Accession Initialization

- Vertebrate + Virome + Mammalian


In [ ]:
cd $WORK

aws s3 cp s3://lovelywater2/sra/viro_SraRunInfo.csv.gz ./
aws s3 cp s3://lovelywater2/sra/vert_SraRunInfo.csv.gz ./
aws s3 cp s3://lovelywater2/sra/mamm_SraRunInfo.csv.gz ./

gzip -d *

In [ ]:
head -n1 mamm_SraRunInfo.csv > test_SraRunInfo.csv
grep 'SRR2136906' mamm_SraRunInfo.csv >> test_SraRunInfo.csv
grep 'ERR2756788' mamm_SraRunInfo.csv >> test_SraRunInfo.csv
grep 'SRR8890821' mamm_SraRunInfo.csv >> test_SraRunInfo.csv

BATCH='test_SraRunInfo.csv'

wc -l  *
md5sum *

### Terraform Initialize

In [11]:
# For rapid batching; copy out serratus folder
TF=$SERRATUS/terraform/main
cd $TF
git diff main.tf
terraform init

# Launch Terraform Cluster
# Initialize the serratus cluster with minimal nodes
terraform apply -auto-approve



diff --git a/terraform/main/main.tf b/terraform/main/main.tf
index de2d00d..7bba7e2 100644
--- a/terraform/main/main.tf
+++ b/terraform/main/main.tf
@@ -92,7 +92,7 @@ module "scheduler" {
   
   security_group_ids = [aws_security_group.internal.id]
   key_name           = var.key_name
-  instance_type      = "c5.large"
+  instance_type      = "r5.2xlarge"
   dockerhub_account  = var.dockerhub_account
   scheduler_port     = var.scheduler_port
 }
@@ -105,7 +105,7 @@ module "monitoring" {
   key_name           = var.key_name
   scheduler_ip       = module.scheduler.private_ip
   dockerhub_account  = var.dockerhub_account
-  instance_type      = "r5.large"
+  instance_type      = "r5.2xlarge"
 }
 
 // Serratus-dl
@@ -113,13 +113,13 @@ module "download" {
   source             = "../worker"
 
   desired_size       = 0
-  max_size           = 200
+  max_size           = 5000
 
   dev_cidrs          = var.dev_cidrs
   security_group_ids = [aws_security_group.internal.id]
 
-  instance_type

In [23]:
# For rapid batching; copy out serratus folder -- round 2 mammalian
TF=$SERRATUS/terraform/main
cd $TF
git diff main.tf
terraform init

# Launch Terraform Cluster
# Initialize the serratus cluster with minimal nodes
terraform apply -auto-approve



diff --git a/terraform/main/main.tf b/terraform/main/main.tf
index de2d00d..63fc863 100644
--- a/terraform/main/main.tf
+++ b/terraform/main/main.tf
@@ -92,7 +92,7 @@ module "scheduler" {
   
   security_group_ids = [aws_security_group.internal.id]
   key_name           = var.key_name
-  instance_type      = "c5.large"
+  instance_type      = "r5.2xlarge"
   dockerhub_account  = var.dockerhub_account
   scheduler_port     = var.scheduler_port
 }
@@ -105,7 +105,7 @@ module "monitoring" {
   key_name           = var.key_name
   scheduler_ip       = module.scheduler.private_ip
   dockerhub_account  = var.dockerhub_account
-  instance_type      = "r5.large"
+  instance_type      = "r5.2xlarge"
 }
 
 // Serratus-dl
@@ -113,13 +113,13 @@ module "download" {
   source             = "../worker"
 
   desired_size       = 0
-  max_size           = 200
+  max_size           = 5000
 
   dev_cidrs          = var.dev_cidrs
   security_group_ids = [aws_security_group.internal.id]
 
-  instance_type

In [24]:
cd $TF

# Open SSH tunnels to the monitor
./create_tunnels.sh

# If you get an error on port
# run:
# ps aux | grep ssh
# sudo kill <PID of SSH>



Tunnels created:
    localhost:3000 = grafana
    localhost:9090 = prometheus
    localhost:5432 = postgres
    localhost:8000 = scheduler


In [7]:
BATCH='test_SraRunInfo.csv'
wc -l $WORK/$BATCH

4 /home/artem/serratus/notebook/200706B_ab/test_SraRunInfo.csv


In [8]:
# Upload SraRunInfo.csv into Serratus
cd $TF
./uploadSRA.sh $WORK/$BATCH

Loading SRARunInfo into scheduler 
  File: /home/artem/serratus/notebook/200706B_ab/test_SraRunInfo.csv
  date: Sat Jul 11 01:41:54 PDT 2020
  wc  : 4 /home/artem/serratus/notebook/200706B_ab/test_SraRunInfo.csv
  md5 : 5df06edfb80cc6dfbab17e4ae0358e2a  /home/artem/serratus/notebook/200706B_ab/test_SraRunInfo.csv


--------------------------
tmp.chunk00
4 tmp.chunk00_sraRunInfo.csv
2943e01a1460e6bfbc0caf1efc5257ad  tmp.chunk00_sraRunInfo.csv
{"inserted_rows":3,"total_rows":3}


 uploadSRA complete.


In [14]:
BATCH='viro_SraRunInfo.csv'
wc -l $WORK/$BATCH
md5sum $WORK/$BATCH

22252 /home/artem/serratus/notebook/200706B_ab/viro_SraRunInfo.csv
e9222b54cee8a65bc3781589f5cbf642  /home/artem/serratus/notebook/200706B_ab/viro_SraRunInfo.csv


In [15]:
# Upload SraRunInfo.csv into Serratus
cd $TF
./uploadSRA.sh $WORK/$BATCH

Loading SRARunInfo into scheduler 
  File: /home/artem/serratus/notebook/200706B_ab/viro_SraRunInfo.csv
  date: Sat Jul 11 09:34:22 PDT 2020
  wc  : 22252 /home/artem/serratus/notebook/200706B_ab/viro_SraRunInfo.csv
  md5 : e9222b54cee8a65bc3781589f5cbf642  /home/artem/serratus/notebook/200706B_ab/viro_SraRunInfo.csv


--------------------------
tmp.chunk00
10001 tmp.chunk00_sraRunInfo.csv
1c849e2aa650e7ffb6054368d63d690e  tmp.chunk00_sraRunInfo.csv
{"inserted_rows":10000,"total_rows":10000}
--------------------------
tmp.chunk01
10001 tmp.chunk01_sraRunInfo.csv
ccddf45754ff78d7f8cb4c2d7decca4a  tmp.chunk01_sraRunInfo.csv
{"inserted_rows":10000,"total_rows":20000}
--------------------------
tmp.chunk02
2252 tmp.chunk02_sraRunInfo.csv
2cd1ed3a5b70fa0d3478734a439ef9c4  tmp.chunk02_sraRunInfo.csv
{"inserted_rows":2250,"total_rows":22250}


 uploadSRA complete.


In [16]:
BATCH='vert_SraRunInfo.csv'
wc -l $WORK/$BATCH
md5sum $WORK/$BATCH

94909 /home/artem/serratus/notebook/200706B_ab/vert_SraRunInfo.csv
e39b50b78465f7e12676ef18d179de5f  /home/artem/serratus/notebook/200706B_ab/vert_SraRunInfo.csv


In [17]:
# Upload SraRunInfo.csv into Serratus
cd $TF
./uploadSRA.sh $WORK/$BATCH

Loading SRARunInfo into scheduler 
  File: /home/artem/serratus/notebook/200706B_ab/vert_SraRunInfo.csv
  date: Sat Jul 11 12:33:37 PDT 2020
  wc  : 94909 /home/artem/serratus/notebook/200706B_ab/vert_SraRunInfo.csv
  md5 : e39b50b78465f7e12676ef18d179de5f  /home/artem/serratus/notebook/200706B_ab/vert_SraRunInfo.csv


--------------------------
tmp.chunk00
10001 tmp.chunk00_sraRunInfo.csv
0d150dff3e0ac0cd889f63dda882fc68  tmp.chunk00_sraRunInfo.csv
{"inserted_rows":10000,"total_rows":32250}
--------------------------
tmp.chunk01
10001 tmp.chunk01_sraRunInfo.csv
c103d98631ff78fdb4d239442b1af6c2  tmp.chunk01_sraRunInfo.csv
{"inserted_rows":10000,"total_rows":42250}
--------------------------
tmp.chunk02
10001 tmp.chunk02_sraRunInfo.csv
30faebcec876ba6ded6cc6e60c9230b5  tmp.chunk02_sraRunInfo.csv
{"inserted_rows":10000,"total_rows":52250}
--------------------------
tmp.chunk03
10001 tmp.chunk03_sraRunInfo.csv
6679c2e81960c4c5b77bfa76c679702c  tmp.chunk03_sraRunInfo.csv
{"inserted_rows":1

In [25]:
BATCH='mamm_SraRunInfo.csv'
wc -l $WORK/$BATCH
md5sum $WORK/$BATCH

100799 /home/artem/serratus/notebook/200706B_ab/mamm_SraRunInfo.csv
499fa3d5a1fa8cf86efce1925c7e27fd  /home/artem/serratus/notebook/200706B_ab/mamm_SraRunInfo.csv


In [27]:
# Upload SraRunInfo.csv into Serratus
cd $TF
./uploadSRA.sh $WORK/$BATCH

Loading SRARunInfo into scheduler 
  File: /home/artem/serratus/notebook/200706B_ab/mamm_SraRunInfo.csv
  date: Sat Jul 11 19:39:03 PDT 2020
  wc  : 100799 /home/artem/serratus/notebook/200706B_ab/mamm_SraRunInfo.csv
  md5 : 499fa3d5a1fa8cf86efce1925c7e27fd  /home/artem/serratus/notebook/200706B_ab/mamm_SraRunInfo.csv


--------------------------
tmp.chunk00
10001 tmp.chunk00_sraRunInfo.csv
7aa82113805577ffd6ba62ac1ee9fe6e  tmp.chunk00_sraRunInfo.csv
{"inserted_rows":10000,"total_rows":10000}
--------------------------
tmp.chunk01
10001 tmp.chunk01_sraRunInfo.csv
f7acae754c6faa344f439e8979686eb5  tmp.chunk01_sraRunInfo.csv
{"inserted_rows":10000,"total_rows":20000}
--------------------------
tmp.chunk02
10001 tmp.chunk02_sraRunInfo.csv
e585284b4d0ee0e7880f78d52716e241  tmp.chunk02_sraRunInfo.csv
{"inserted_rows":10000,"total_rows":30000}
--------------------------
tmp.chunk03
10001 tmp.chunk03_sraRunInfo.csv
b59dc3b33eb38f1d2b2a98dc08755914  tmp.chunk03_sraRunInfo.csv
{"inserted_rows":

## Run Serratus

In [30]:
# Set Cluster Parameters =============================
## get Config File (if it doesn't exist)
# curl localhost:8000/config | jq > serratus-config.json
#
cd $TF
# Make local changes to config file
echo "  Cluster Config File: "
cat serratus-config.json
echo ""
echo ""
# Re-upload config file
curl -T serratus-config.json localhost:8000/config

  Cluster Config File: 
{
  "ALIGN_ARGS": "--very-sensitive-local",
  "ALIGN_MAX_INCREASE": 50,
  "ALIGN_SCALING_CONSTANT": 0.025,
  "ALIGN_SCALING_ENABLE": true,
  "ALIGN_SCALING_MAX": 0,
  "CLEAR_INTERVAL": 600000,
  "DL_ARGS": "",
  "DL_MAX_INCREASE": 10,
  "DL_SCALING_CONSTANT": 0.1,
  "DL_SCALING_ENABLE": true,
  "DL_SCALING_MAX": 0,
  "GENOME": "protref3",
  "MERGE_ARGS": "protein",
  "MERGE_MAX_INCREASE": 25,
  "MERGE_SCALING_CONSTANT": 0.1,
  "MERGE_SCALING_ENABLE": true,
  "MERGE_SCALING_MAX": 60,
  "SCALING_INTERVAL": 120,
  "VIRTUAL_SCALING_INTERVAL": 35
}

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0{"ALIGN_ARGS":"--very-sensitive-local","ALIGN_MAX_INCREASE":50,"ALIGN_SCALING_CONSTANT":0.025,"ALIGN_SCALING_ENABLE":true,"ALIGN_SCALING_MAX":0,"CLEAR_INTERVAL":600000,"DL_ARGS":"","DL_MAX_

### Error handling

In [ ]:
## Stop postgres if it's running 
# systemctl stop postgresql

## Connect to postgres
# psql -h localhost postgres postgres

#  psql -h localhost postgres postgres -c "DELETE FROM blocks WHERE state = 'done';"

### ACCESSION OPERATIONS
## Reset SPLITTING accessions to NEW
# UPDATE acc SET state = 'new' WHERE state = 'splitting';

## Reset SPLIT_ERR accessions to NEW
## (repeated failures can be missing SRA data)
# UPDATE acc SET state = 'new' WHERE state = 'split_err';

## Reset MERGE_ERR accessions to SPLIT_DONE
# UPDATE acc SET state = 'split_done' WHERE state = 'merge_err';

## Clear DONE Accessions (ONLY ON COMPLETION)
# DELETE FROM acc WHERE state = 'merge_done';

### BLOCK OPERATIONS

##  Reset FAIL blocks to NEW
# UPDATE blocks SET state = 'new' WHERE state = 'fail';

# Reset ALIGNING blocks to NEW
# UPDATE blocks SET state = 'new' WHERE state = 'aligning';

# Clear Done
# DELETE FROM blocks WHERE state = 'done';

# RESET STATE
# DELETE FROM blocks WHERE state = 'done';
# DELETE FROM blocks WHERE state = 'fail';
#
#
# DELETE FROM acc WHERE state = 'split_err';
# DELETE FROM acc WHERE state = 'merging';
# DELETE FROM acc WHERE state = 'merge_err';
# DELETE FROM acc WHERE state = 'split_done';


In [ ]:
# Nuke Shutdown
aws ec2 describe-instances \
  --filter Name=tag:Name,Values=serratus-align-instance \
  > align_instances.json

jq '.Reservations[].Instances[].InstanceId' -r align_instances.json \
  | pv -l \
  | xargs -n10 -P10 aws ec2 terminate-instances --instance-ids

In [ ]:
### UPDATE TO PROTREF CAME THROUGH
# SHUTTING DOWN AND RESTARTING AFTER 34K


## CoV Shortlist

Filter output from .psummary as:

CoV_id95-60_reads100p_score30p.txt


SRR5447167
SRR4051841
SRR4051861
SRR7734135
SRR8735152
SRR1033849
SRR1104919
SRR8335010
ERR2737743
ERR3180650
ERR2737744
ERR3180736
ERR1301857
ERR1301942
SRR1104917
SRR1104916
ERR2738465
ERR3180645
ERR2738466
ERR3180731
SRR7239978
SRR1104918
ERR1301501
ERR1301515
ERR1301590
ERR1301604
SRR6713795
SRR3135336
SRR10873981
SRR11825060
SRR3135330
SRR4051862
ERR1301465
ERR1301554
SRR6713701
SRR8335004
SRR8379618
SRR5333364
SRR5078288
ERR1301521
ERR1301610
SRR10873977
SRR5333370
SRR7734139
SRR9112931
SRR5431762
SRR5333359
SRR6450836
SRR5447160
SRR3135321
SRR6713707
SRR5333371
SRR5431764
SRR5431765
SRR5333369
SRR5801719
ERR1301463
SRR5333382
SRR10168377
ERR1301552
SRR6367407
SRR5333373
SRR5801720
SRR5333377
SRR5333381
SRR5333368
SRR5801718
SRR5333380
SRR8296271
SRR5333360
SRR5447158
SRR5343976
SRR10058155
SRR5415517
ERR1301477
SRR5447156
ERR1301566
ERR1303041
SRR5040919
SRR8296269
SRR5333375
SRR5333367
SRR10168378
SRR5333376
SRR8736196
SRR7287609
ERR2013702
SRR5447172
SRR8335007
SRR5801725
SRR8833174
SRR5333379
SRR10540651
SRR6450838
ERR1301471
SRR5333378
SRR8304495
SRR8296268
SRR6788807
SRR5210464
ERR1301560
SRR8296270
SRR5801721
ERR1301462
SRR8667359
ERR1301551
SRR5801722
SRR5447163
SRR6450837
ERR1301448
ERR1301469
ERR1301558
ERR1301537
SRR5333374
SRR5801723
SRR7139972
SRR5141036
SRR7239989
SRR10268283
SRR8113437
SRR10875068
SRR8416116
SRR8735376
SRR7239986
SRR7239988
SRR8296267
SRR8735373
SRR8736211
SRR7139958
SRR5431763
SRR6179331
SRR057952
SRR11126663
SRR5447135
SRR11826178
ERR1301530
SRR6311475
SRR5447146
SRR7139959
ERR1301619
SRR8735372
SRR8109023
SRR8735371
ERR1302946
SRR9841161
SRR1522982
SRR11826186
SRR10112123
SRR5801724
SRR8304478
SRR11126662
SRR10268259
SRR11798650
SRR8953105
ERR2013698
ERR2738384
ERR3179763
SRR5447138
SRR8953106
ERR2738383
ERR3179727
SRR10549457
SRR5447152
SRR7186851
SRR6322454
SRR3578151
SRR594501
SRR8953109
SRR10268256
SRR9841163
SRR8736200
SRR5447166
SRR8736149
SRR5447173
ERR1301470
SRR10268258
SRR8991387
SRR6788789
SRR10112126
SRR5447159
SRR3458562
ERR1301559
SRR6788788
SRR10268293
SRR9841160
SRR6325465
SRR8953102
SRR6788775
SRR10112125
SRR3578143
SRR8304479
SRR5431753
SRR7139952
SRR8307566
SRR9915303
ERR3132335
SRR10549470
SRR5447148
ERR1301602
ERR1301494
SRR7820574
ERR1301513
ERR1301583
SRR8736187
SRR7524591
ERR2756788
SRR8736188
SRR5040920
SRR9841168
SRR5210461
SRR8304483
SRR8325796
SRR9112936
SRR6713794
SRR7139949
SRR8304493
SRR5447147
SRR6788791
ERR1301490
SRR8304476
SRR8304477
SRR9841174
SRR9419516
SRR9673760
SRR5447175
SRR8953115
SRR8953114
ERR1301828
SRR10268294
ERR2013700
ERR1301913
SRR5644066
SRR7031407
ERR1301485
SRR7139947
ERR1301574
SRR8833169
SRR7139948
SRR8736192
SRR6713796
SRR5447154
SRR3029229
SRR3030847
ERR3132345
SRR9841173
SRR2295487
SRR6798415
ERR3132336
SRR5447150
SRR9673772
SRR7495348
SRR1562248
SRR10268291
SRR7263050
SRR8953117
SRR7817240
SRR8667356
SRR10010053
SRR057939
SRR8293419
SRR3458565
SRR10112129
SRR8304486
SRR10268286
SRR8833171
SRR5121398
ERR1301833
SRR10112124
SRR7699540
SRR7031405
SRR5447161
ERR2013697
SRR5078300
SRR5943895
ERR1301918
SRR9841176
SRR8983108
SRR8736170
SRR6788787
SRR8270369
SRR8736177
SRR7263035
SRR3458563
SRR9841167
SRR5415515
SRR8735370
SRR8953113
SRR5136500
SRR10436229
SRR7263049
SRR9822079
SRR6713735
SRR9419518
ERR1301567
SRR7617956
SRR6179394
SRR8736209
SRR5415516
SRR8667358
SRR9419521
ERR3132334
ERR3132338
SRR9644977
ERR2013696
SRR11267837
SRR6173479
SRR8953110
SRR7699536
SRR8304488
SRR8297844
SRR5447168
SRR10436233
ERR1301484
SRR5319795
SRR9419515
SRR5447134
SRR8735374
SRR9841175
SRR8953107
ERR1301573
SRR8464529
SRR6788792
SRR2259813
ERR2013694
ERR3994223
SRR8953118
SRR9673276
SRR8464525
SRR8304487
SRR9419523
SRR8953103
SRR9822080
SRR9915302
ERR3132344
SRR8736152
SRR7031408
SRR9841177
SRR7699538
ERR3179189
SRR8464527
SRR7263047
SRR8109019
SRR11241423
SRR10268271
SRR5415514
SRR9841172
SRR6175505
ERR1301889
SRR7699543
SRR5431741
ERR1301804
SRR9650564
SRR6227595
SRR8464528
SRR7031420
SRR8736198
SRR4204943
SRR7031425
ERR1298640
SRR7495345
SRR5447169
SRR8464526
SRR6956994
SRR8736171
SRR8953108
SRR9640795
SRR11267831
SRR10100472
SRR7637209
SRR316295
SRR5431738
SRR9667032
SRR9650565
SRR6227598
SRR7617951
SRR057858
SRR5590997
SRR8464531
SRR8304489
SRR8139282
SRR8304506
SRR9967677
SRR8953112
SRR8953120
SRR9841170
SRR5136504
SRR11060135
SRR8464533
SRR7617955
SRR8833173
SRR10100474
ERR2738550
ERR3179341
SRR5888870
SRR5447149
SRR6926009
SRR5431743
SRR8953111
SRR11267834
SRR6925945
SRR6706122
SRR8618343
SRR2264320
SRR9841169
ERR2738549
ERR3179246
SRR11798649
SRR2264321
SRR8116664
SRR8143991
ERR3132343
SRR8116663
SRR3578141
SRR7617959
SRR5136505
SRR5447139
SRR7097939
SRR5381491
SRR7239364
SRR6180838
SRR5040910
SRR6706120
SRR8618353
SRR8464532
SRR8489958
SRR7495346
SRR7263039
DRR124160
SRR5381493
SRR11349394
SRR11267835
SRR5210373
SRR7206533
SRR11068453
SRR7811372
SRR7239363
SRR6180842
SRR7495350
SRR7817239
ERR1994246
SRR5210462
SRR7699537
SRR1422851
SRR7829378
SRR7495349
SRR3319299
SRR8304481
SRR7811378
SRR7311308
SRR2259814
SRR6227590
SRR057955
SRR8416105
SRR5136503
SRR7705519
SRR6713800
SRR9915312
SRR6706117
SRR8618347
SRR10100473
SRR9960038
SRR5431766
SRR7791680
SRR9915311
SRR8304504
SRR7311311
SRR10100470
SRR8304482
SRR10010054
ERR1994252
SRR10466909
SRR8198098
SRR7647049
ERR3277161
SRR8464530
SRR10867450
SRR9960043
SRR8953116
SRR11128869
SRR9112926
SRR10058578
SRR10466907
SRR5447178
SRR7263040
SRR9915309
SRR5040900
SRR8116672
SRR1502195
SRR1502198
SRR1502197
SRR6925941
SRR9419522
ERR3277153
SRR9915313
SRR11267836
SRR6325463
SRR6925973
SRR7791699
SRR7791687
SRR9089148
SRR5447136
SRR11241424
SRR7495347
SRR7495341
SRR6925965
SRR7617954
SRR6227597
SRR8379623
SRR8736168
SRR5188766
SRR6925976
SRR6227599
SRR8489959
SRR7791676
SRR6926000
SRR11183281
SRR7791694
SRR7791691
SRR6820460
SRR8389794
ERR2737675
ERR3179731
SRR9733998
SRR11183280
SRR6706115
SRR8618350
ERR1301855
SRR5447151
SRR6925946
ERR2737676
ERR3179767
SRR6925962
SRR8389791
SRR7811367
SRR9960044
ERR3994143
SRR10100471
SRR5447153
SRR8335006
SRR6706125
SRR8618349
SRR6325460
SRR6227591
SRR9211912
ERR1301940
SRR6381605
SRR7811382
ERR2738457
ERR3180649
SRR10001349
SRR8302132
SRR6925944
DRR124162
ERR2738458
ERR3180735
SRR6227594
SRR8109025
SRR6925967
SRR6834163
SRR11318186
SRR6227593
SRR10897353
ERR1301603
SRR7524643
SRR7895177
SRR10897359
ERR2737561
ERR3179988
SRR7263046
SRR7810962
SRR6925974
SRR4787434
SRR10867442
SRR8735375
ERR1301514
SRR6925959
ERR2737562
ERR3180076
SRR7791689
SRR1033850
SRR8416115
SRR9915308
SRR7791686
SRR7495340
SRR9915304
SRR6322456
SRR7791674
SRR6008299
SRR6925964
SRR6820457
SRR5447171
SRR7791682
SRR6713747
SRR7791690
SRR7791692
SRR5447140
SRR7791667
SRR7811381
SRR7031417
SRR10058577
SRR6713768
SRR8053814
SRR8189301
SRR7791681
SRR10010050
SRR8265623
SRR7811368
SRR7339902
SRR8720654
SRR7811383
DRR169709
SRR6374478
SRR8717238
SRR8389849
SRR6834166
SRR7791670
SRR9915307
SRR8109027
SRR6325464
SRR7699542
ERR1691914
SRR9640796
ERR1301589
SRR7641103
SRR7811379
SRR3496934
SRR10466906
SRR6925942
SRR7811371
SRR10436242
SRR5210465
SRR8416119
SRR8489954
SRR8717234
SRR8389847
SRR6926004
SRR10897350
ERR1301500
SRR7495336
SRR8489960
SRR6325461
SRR6381571
ERR2013701
SRR9915305
SRR7631508
ERR1994258
SRR9640793
SRR10010051
SRR8053824
SRR10100469
ERR3132337
SRR8416118
SRR7587551
ERR1301898
SRR9915306
SRR5447142
ERR2738380
ERR3179758
SRR6925952
SRR6713771
SRR7791685
SRR11486166
SRR11522605
ERR2013695
SRR10436230
SRR8953119
SRR5813968
SRR7817241
SRR5040921
ERR1301813
SRR6820463
SRR5888876
SRR8053813
SRR6820458
ERR2738379
ERR3179722
SRR7522706
SRR6925956
SRR2990992
SRR8132768
ERR3132328
SRR7632682
SRR6926001
SRR6713798
SRR5447133
SRR8389793
ERR3132333
SRR5888859
SRR8109016
SRR6820465
ERR3132330
SRR5381496
SRR7895178
SRR6925957
SRR5447155
SRR8053816
SRR7268697
SRR6820464
SRR4417125
SRR6322457
SRR10466920
SRR7820575
SRR11278414
SRR7617952
SRR7586646
SRR6925980
SRR8116671
SRR8953101
SRR8053815
ERR2738374
ERR3179765
SRR7263036
SRR7637206
SRR11318190
SRR10436226
SRR8053823
ERR3180614
ERR1301596
SRR4253225
ERR1301584
SRR6820462
ERR2639616
SRR6820459
ERR2738373
ERR3179729
SRR9855987
SRR7522705
SRR6926006
ERR3180530
SRR6925969
SRR11040411
SRR6925953
SRR9672927
SRR5818205
SRR9089146
SRR11798648
SRR8053822
SRR11278415
SRR8389835
SRR9967676
SRR4417128
SRR5888878
SRR9733996
SRR8335005
SRR11486167
SRR11522606
SRR6788781
SRR8717233
SRR7632627
SRR6925954
SRR7263045
SRR6925993
SRR5431760
SRR7522703
ERR1749789
SRR9640819
SRR10058572
SRR10436210
SRR5590996
ERR2576518
SRR5412251
SRR10867447
SRR11040413
SRR8265622
SRR5078299
SRR5867219
SRR9831606
SRR8198097
SRR11278424
SRR6007415
ERR3994141
SRR5040902
ERR1301904
SRR8953104
SRR5888879
ERR1303043
SRR7031428
SRR8389832
SRR10436238
SRR6375802
SRR5078301
SRR7031404
SRR8717237
SRR8389837
SRR6925972
SRR5867223
SRR11040408
ERR2013699
SRR11278420
SRR9640807
ERR2843943
ERR3132342
SRR6788786
SRR7263042
SRR10873780
DRR124159
SRR4417126
SRR6925989
SRR6925996
SRR6478574
SRR9159932
SRR11278419
SRR11458633
SRR1422850
SRR9089150
SRR5260688
SRR11278417
SRR5381486
SRR7524641
SRR11278422
SRR8198093
SRR5914001
SRR11318191
SRR5447145
SRR8265592
SRR8389831
ERR2738371
ERR3179730
SRR4417127
SRR6926007
SRR11318184
SRR11278416
SRR8265594
SRR7632683
SRR5867221
SRR6925983
ERR2738372
ERR3179766
SRR11278418
SRR9665480
SRR5210374
SRR7263038
SRR10436244
ERR1301499
SRR6825928
SRR5210372
SRR1324965
SRR9831613
SRR10097302
ERR2001076
ERR3132329
SRR8132764
SRR5888874
SRR6411046
ERR2001083
ERR2639610
SRR7244716
SRR10097304
SRR8936995
SRR9831611
SRR6478570
SRR6925961
SRR8949238
SRR3578139
SRR7522701
ERR1301879
SRR5188765
SRR6925963
ERR1301794
SRR11318188
SRR9831614
SRR5381488
SRR8416107
SRR9089141
SRR5381485
SRR6925979
SRR8776456
SRR11522598
SRR11486170
SRR6925990
ERR3180396
SRR5914000
SRR11613372
SRR11250207
SRR5098029
SRR10867440
SRR11486165
SRR11522604
SRR5823235
DRR169714
ERR3180310
SRR10436215
SRR6925984
ERR3132339
ERR1301856
SRR8416117
SRR7263041
ERR1301822
SRR11610892
SRR6788785
ERR3180491
ERR3180575
SRR8523631
SRR6926003
SRR6168955
ERR1301941
ERR2639618
SRR6925948
ERR1994199
SRR6925947
SRR11610895
ERR1302963
SRR9077336
SRR8109028
SRR7885854
SRR6478576
SRR6926012
SRR8109015
SRR8389822
SRR8389821
SRR10257208
SRR5365809
SRR5997671
SRR6925958
SRR6925994
DRR083611
SRR5575205
SRR8381861
SRR6834162
SRR8132766
SRR8416114
SRR5913992
ERR3849998
SRR7522702
SRR7749872
DRR169711
SRR6980960
SRR8416112
SRR8132769
ERR2639611
SRR8582179
SRR8821851
ERR1301509
SRR5913998
SRR5447137
SRR11613376
SRR1515192
SRR2991007
SRR6925995
SRR6181335
SRR6478567
ERR1301854
SRR9211911
ERR1301840
DRR124163
SRR5210375
SRR5888886
ERR1301598
SRR11613373
SRR5864116
SRR10423918
SRR8113436
SRR5098025
SRR8389823
ERR1301925
SRR5888877
SRR6926013
ERR1301443
SRR6825302
SRR11613368
SRR9831609
SRR6925997
SRR9089144
ERR1301778
SRR11486168
SRR11522597
SRR7268698
SRR10875097
DRR083612
ERR1301861
SRR7782578
SRR5823234
ERR1301863
SRR8582183
SRR8821854
SRR5381487
ERR1301532
SRR8116674
SRR8379615
SRR5867217
DRR169717
SRR9831608
DRR169716
SRR9831607
SRR5098023
SRR3496930
SRR5381497
SRR10058573
SRR8379614
ERR2639613
SRR413774
SRR6701618
ERR3180546
ERR2737563
ERR3179991
SRR5098022
SRR6181332
SRR11458626
ERR1301786
SRR6925977
SRR6925981
SRR2418554
ERR3180630
ERR2737564
ERR3180079
SRR9098033
DRR124166
SRR5575204
SRR11613367
SRR5888881
SRR11613378
ERR1994249
ERR2738382
ERR3179764
SRR5665758
SRR10423914
DRR083610
SRR5383919
SRR8132761
SRR8523632
DRR124169
ERR2737742
ERR3180737
SRR10058576
SRR10423909
SRR5381490
SRR6337032
SRR10867436
SRR9672953
ERR2738375
ERR3179724
ERR1301871
SRR9831621
DRR169712
ERR2738556
ERR3179336
SRR7244683
SRR7522704
SRR8582182
SRR8821852
SRR8109022
ERR2737741
ERR3180651
SRR11613369
ERR2738376
ERR3179760
SRR8840676
SRR11613374
ERR2684378
ERR2738386
ERR3179762
SRR8416106
ERR3180347
SRR6235259
SRR5381498
SRR8109029
DRR169710
ERR2738381
ERR3179728
ERR2030034
SRR6478569
SRR5098028
ERR3849999
ERR1302508
SRR8132763
ERR2738555
ERR3179241
SRR8389825
ERR3180433
SRR11613371
SRR5888861
SRR10058575
DRR169713
SRR10423944
SRR5964611
SRR7239368
SRR10867434
SRR10423897
SRR5823232
SRR6478571
SRR6926016
SRR10423936
ERR2029961
SRR6178088
ERR2738385
ERR3179726
SRR5913990
SRR8113438
ERR1303011
SRR5040906
SRR5823233
SRR8416111
SRR5913989
SRR9665486
SRR10873928
SRR7522699
ERR1302935
SRR10058574
SRR6925943
SRR6008298
SRR10058579
SRR5888875
SRR5383920
SRR5040916
SRR9843166
SRR7522698
SRR9669908
SRR10423919
SRR6788780
SRR9089147
SRR10874916
SRR8197050
SRR11108107
SRR5864115
SRR7782573
SRR9669702
SRR6788779
DRR124167
DRR169708
ERR3180604
SRR10423946
ERR2001717
ERR3994142
ERR1303016
DRR169715
SRR10423942
SRR8189297
SRR11458627
SRR8116676
SRR7885852
SRR8381829
ERR3180520
SRR7495225
SRR6701616
SRR8772199
SRR5868319
SRR6701620
SRR2991001
SRR10423925
SRR8416109
SRR10867437
SRR10058580
SRR10423948
SRR8168890
SRR9843170
SRR7885853
SRR8663603
SRR6178098
ERR1301842
SRR2991009
SRR5997684
SRR8416113
SRR9843171
ERR1301835
SRR8197051
SRR8490248
SRR6926008
SRR5964508
ERR441003
SRR6925992
SRR5913995
SRR8189300
SRR6178095
SRR2990991
SRR8168892
SRR8772195
SRR9089143
DRR083609
SRR9843165
SRR10867449
ERR2214810
SRR7263037
SRR10423895
SRR6701619
SRR9336424
ERR1301564
SRR6478577
SRR6788810
SRR306717
SRR8139277
SRR10867433
SRR8197052
SRR2990988
ERR3850005
SRR5098026
SRR6478568
SRR9831618
ERR1854455
SRR7699541
SRR8949234
ERR2649624
ERR2737536
SRR2990999
SRR8862749
DRR169706
SRR9967683
SRR5381484
SRR6788809
SRR8582176
SRR8821846
SRR5914003
SRR10423906
SRR10423949
SRR8334108
SRR10423907
ERR2001644
SRR8189299
SRR9672750
SRR10522323
SRR10867444
SRR6178093
SRR8109026
SRR5888853
ERR1301616
SRR10423924
SRR11613377
SRR6713821
SRR10423926
ERR2001716
SRR10423933
SRR8936997
SRR8953735
ERR1301527
SRR10423932
SRR2968493
SRR6926010
SRR9089142
SRR5575202
SRR8379616
SRR8772194
SRR5913996
SRR5040912
ERR2639612
DRR083608
SRR5868318
SRR7782575
SRR9672861
SRR10867446
SRR8582177
SRR8821849
SRR057859
ERR1302937
SRR10423935
SRR7184467
SRR8840675
ERR1298571
SRR8416110
SRR8936996
SRR8936990
SRR10423929
SRR10436220
SRR6701611
SRR6713899
SRR8663627
ERR1298575
SRR10423898
ERR1298576
SRR10867441
SRR6178102
SRR8862750
SRR7601545
ERR1298572
SRR10412153
ERR1298570
SRR8772202
SRR5381489
SRR6478566
SRR7601544
SRR5914002
ERR1298574
ERR1298573
ERR1303013
ERR2202464
SRR9843175
SRR9843169
SRR6701613
SRR10423928
SRR11355548
SRR5575199
SRR5964612
ERR1302952
SRR5665768
SRR6701610
SRR10867445
SRR8379608
SRR8416108
SRR5964610
SRR10412152
SRR5913993
SRR6701612
ERR1301575
SRR6478575
ERR1301486
SRR6265746
SRR9843174
SRR10867448
SRR6257762
ERR1298577
ERR1303003
SRR6701614
DRR083613
SRR2516435
SRR6666113
ERR1301812
SRR5913994
ERR1301878
SRR7782585
SRR8776455
SRR6917570
SRR8772201
ERR2737664
ERR3179773
ERR3850003
SRR5665765
SRR7287177
SRR8526186
ERR2214792
SRR8776457
ERR2737663
ERR3179737
ERR2001409
ERR2202463
ERR2001341
SRR5247577
SRR8296266
SRR9672218
SRR7184466
SRR10752301
SRR8304511
ERR1301897
SRR11355549
SRR5823227
SRR2976063
SRR8450046
SRR8293696
ERR1302927
SRR8304498
SRR10412151
SRR5913997
SRR5964509
SRR8304494
SRR5964609
SRR11613370
SRR6713783
ERR2737607
ERR3179286
ERR3180002
SRR057824
SRR6706119
SRR8618352
ERR3179191
SRR11613375
DRR083607
SRR6942720
ERR2214791
SRR3578144
SRR6265752
SRR3225584
SRR3303824
ERR2649538
ERR2737535
SRR5823229
SRR057956
SRR6265743
SRR8398035
SRR6942723
ERR2843944
SRR8772196
SRR5575193
SRR9843172
SRR8304485
SRR5964510
SRR6176024
SRR8582181
SRR8821853
SRR2990983
SRR11355552
SRR6364569
SRR7782571
SRR6378063
SRR7006121
SRR7268700
ERR1854456
SRR6942722
SRR5351754
SRR11262330
ERR2737909
ERR3179965
DRR047248
SRR6706124
SRR8304500
SRR8618348
SRR8490249
SRR6942719
ERR2029975
ERR2030058
SRR7533310
SRR8078976
SRR6713736
SRR7006120
SRR5964513
SRR8139279
ERR2029985
ERR3179105
ERR3178736
SRR10412149
SRR8862747
SRR7782581
ERR1301576
ERR1301620
ERR2738557
ERR3179242
SRR6432696
SRR7782586
ERR2737910
ERR3178641
ERR3180058
SRR5964511
SRR8304480
SRR7268099
SRR8840674
ERR3178761
SRR6284156
SRR5204384
SRR7811373
ERR1301557
SRR6701615
ERR1301487
SRR5823228
SRR6942717
SRR7782584
ERR3179010
SRR11355550
ERR1301482
ERR1301571
ERR3178666
SRR6701617
SRR2157429
SRR8265628
SRR6706118
SRR8618351
SRR6706114
SRR7734081
SRR8618346
SRR2990996
SRR8139285
SRR8379388
ERR2738558
ERR3179337
ERR1301531
SRR5964515
SRR7467391
SRR6788808
ERR2738548
ERR3179340
SRR5964514
SRR6942718
SRR6706123
SRR8618344
SRR11355553
DRR169719
ERR2673224
SRR5964507
SRR6265751
SRR5059389
SRR8936991
SRR5964512
SRR11355551
ERR1302971
SRR6713719
SRR11458629
SRR5950165
ERR1303047
SRR11825055
SRR6942724
SRR8078975
SRR7495223
SRR057823
SRR6180843
ERR1301474
ERR1301811
SRR5665762
SRR7533309
SRR8078974
SRR5964506
SRR7184462
SRR8772200
SRR2955011
SRR8617938
ERR2738547
ERR3179245
ERR1302861
SRR7239366
SRR8936993
ERR1301563
SRR7705518
SRR5964505
SRR8707240
SRR9009084
ERR1301528
SRR10412147
SRR1522121
SRR6706116
SRR8618342
ERR2738232
ERR3180100
ERR1301936
ERR2029829
SRR11118089
DRR169718
ERR1301597
ERR2030092
ERR2738231
ERR3180012
ERR2029902
SRR6706121
SRR8618345
ERR1301617
SRR1663422
ERR2737753
ERR3179228
SRR5665760
SRR6180836
ERR1301508
SRR7239365
SRR2418623
SRR8258957
DRR169720
SRR10412150
SRR5062018
ERR3180584
SRR5665759
ERR2737803
ERR3179237
SRR8663616
SRR5984001
ERR1301489
ERR1301896
SRR7006122
SRR9009085
SRR10023921
ERR2030163
ERR2720647
SRR8296265
SRR8379381
ERR1301851
ERR4074567
SRR8617995
SRR10867827
SRR9924868
ERR1301578
ERR2737804
ERR3179332
SRR10412146
ERR1301460
SRR8132770
ERR1301254
ERR2737674
ERR3179768
ERR2737673
ERR3179732
SRR5590990
SRR8936994
SRR2976064
SRR8258956
ERR4074565
ERR1302961
SRR8379382
SRR7868718
SRR10412145
SRR10412160
SRR3623213
SRR5381499
SRR7811370
SRR8668297
ERR2737751
ERR3179227
ERR1301312
ERR1301455
SRR7811376
ERR2030105
ERR2737752
ERR3179322
SRR9665482
ERR1301600
SRR8377638
SRR7892477
ERR1301593
SRR8936992
ERR2001388
SRR2805762
ERR1301504
ERR2029784
ERR3180318
SRR7811369
SRR8377645
ERR1301451
ERR1301544
SRR7746967
ERR1303037
ERR2030176
ERR2720648
ERR3180404
ERR4074566
SRR6257761
SRR7811384
SRR11611180
ERR1301540
SRR10291931
SRR7782574
SRR9924865
SRR8377644
ERR1301565
SRR10420953
SRR8143992
SRR8377640
ERR2001320
SRR5796823
SRR7974609
SRR8304492
SRR10294020
ERR2030172
ERR2720644
ERR2738377
ERR3179723
ERR2030198
SRR6242095
SRR8377633
SRR4044089
SRR9924867
SRR057896
SRR9665481
SRR9924863
SRR7868733
SRR9924872
SRR7339903
SRR8265626
SRR057938
SRR10291930
SRR10291925
ERR1301800
SRR7089636
SRR5868317
SRR7942618
ERR2673223
SRR7184463
ERR2030127
SRR11611182
ERR2639617
ERR2738378
ERR3179759
SRR7224141
SRR8377639
ERR2193291
SRR1598978
SRR7503962
SRR057953
SRR8258954
ERR1302027
SRR7224139
ERR1301476
SRR7239370
ERR2030038
SRR10412148
SRR6800489
SRR9665479
SRR4044084
SRR9822078
SRR057914
SRR1662181
SRR6257753
SRR7226349
ERR3180326
SRR10291927
SRR8377636
ERR1301957
ERR2738387
ERR3179725
ERR2030101
ERR3178349
SRR057912
SRR11822959
ERR1301885
ERR2029965
SRR7224140
SRR8281649
SRR057967
ERR1301753
ERR3178254
SRR057940
SRR8868208
SRR5344014
ERR2738388
ERR3179761
SRR1663421
SRR9924866
ERR1302962
SRR10990419
ERR1301526
ERR1301852
SRR057929
ERR1303038
SRR8617998
SRR9924862
ERR2737800
ERR3179330
SRR4045336
ERR1840887
SRR6489330
ERR2737749
ERR3180046
ERR1301615
SRR10867828
SRR7164571
SRR7467394
SRR8053820
SRR7853483
SRR8735278
SRR10873475
SRR2829970
ERR1301044
ERR1303247
ERR2737799
ERR3179235
SRR8181895
ERR1301677
SRR10970200
ERR1301826
SRR9670921
SRR5137777
ERR3180412
SRR6176023
SRR5138014
SRR6257757
SRR7293002
ERR1303248
SRR6257758
ERR2649605
ERR2738202
SRR8437727
ERR1303163
SRR6411055
SRR8181914
ERR2738075
ERR3179356
SRR6489329
SRR8617894
ERR1301785
SRR5059393
SRR6207173
SRR10970244
SRR5137781
ERR2639615
ERR2738076
ERR3179451
SRR8772198
ERR1301888
SRR10023898
SRR057951
SRR7868708
SRR9924870
SRR9924861
ERR2738214
ERR3179494
SRR7868728
ERR1301962
ERR2738213
ERR3179399
ERR2596695
SRR10291923
SRR7127908
SRR7287286
ERR1302712
SRR5137767
ERR1302141
SRR8113262
ERR2030007
SRR5625907
SRR7746960
SRR7947924
SRR2955013
SRR4409397
ERR3180460
SRR8868215
SRR6898026
ERR2001627
SRR6257754
ERR2029901
SRR057897
ERR2001686
ERR3178310
ERR1302220
ERR3994145
SRR8954552
ERR1301452
ERR2029885
ERR2737992
ERR3179780
SRR7164559
SRR7853482
SRR8181915
SRR8735277
SRR8868203
ERR2738554
ERR3179339
SRR4045337
ERR1302467
ERR1302884
ERR1303072
SRR8647628
ERR1301801
SRR5137792
ERR1302795
ERR3179213
ERR2001128
ERR3180374
SRR10028470
SRR8868214
ERR1301989
ERR1302670
ERR2737736
ERR3180739
SRR6384888
ERR1301886
ERR1302370
SRR11601821
ERR1302031
ERR1302124
SRR10566189
SRR11027564
SRR6384890
SRR7235516
SRR6411038
SRR6898042
ERR1302225
ERR1301853
ERR1303075
ERR2737735
ERR3179308
ERR3180653
SRR2290960
SRR5319850
SRR8948423
ERR1301285
ERR1301992
ERR1301938
ERR1302636
ERR1302996
ERR2737991
ERR3179744
SRR6426023
SRR7947919
ERR1301541
ERR1302046
ERR2738553
ERR3179244
SRR6426015
ERR1302471
ERR1302840
SRR8617946
ERR1301227
SRR10358006
SRR7089630
ERR1301788
SRR7899951
ERR2304200
ERR2738443
ERR3179574
ERR3994210
SRR7061300
ERR1302012
ERR2738004
ERR3179782
ERR1302623
ERR1302989
SRR5469044
SRR7089634
ERR1301378
ERR1302602
SRR5803834
ERR1302985
ERR1302984
SRR10042027
ERR1302999
ERR2737790
ERR3179327
SRR5469056
SRR6246041
ERR1302461
ERR1303060
SRR10033018
ERR1302572
ERR3180350
SRR7089633
ERR1301981
ERR1301761
SRR1781596
SRR8675860
ERR1302843
SRR6246044
SRR7061292
ERR1301865
SRR6426014
ERR2001087
SRR9622561
ERR2030160
SRR8181903
SRR11027563
SRR11217070
ERR1303058
SRR6921825
ERR1302447
ERR1302050
ERR1303238
SRR8367575
SRR10522784
SRR5997814